# Demo - Python visualisation libraries

This notebook was used to demo some plotting libraries at BioInfoSummer 2017 and in my [PyConAU 2017 talk](https://www.youtube.com/watch?v=6d3Yk7a2qYI).

Depending on what version of Jupyter you're running, you may need to launch this notebook with a higher data rate limit so that visualisation libraries are not throttled in communicating with the browser, e.g.

`jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000`

This issue is referenced [here](https://github.com/jupyter/notebook/issues/2287).

### Libraries

You can find documentation for libraries used in this notebook at respective home pages:

* [Matplotlib](https://matplotlib.org/)
* [Seaborn](https://seaborn.pydata.org/)
* [Plotly (python API)](https://plot.ly/python/)
* [Bokeh](http://bokeh.pydata.org/en/latest/)
    * Note that the higher-level [bkcharts](https://github.com/bokeh/bkcharts) package mentioned in the PyConAU talk has been deprecated in favour of Holoviews
* [Holoviews](http://holoviews.org/)

I'm relatively inexperienced with some of these libraries, so there may well be better ways to set up some of the plots below! 

And some libraries explicitly discussed, but not demoed:

* [bqplot](https://bqplot.readthedocs.io/en/stable/) - the [examples](https://github.com/bloomberg/bqplot/tree/master/examples) repository is useful
* [Altair](https://altair-viz.github.io/)

### Data

The dataset used in this demo was downloaded from the US [National Electronic Injury Surveillance System](https://www.cpsc.gov/Research--Statistics/NEISS-Injury-Data). For this demo, I used incidents with patient ages 5-80 years, occuring in the month of December 2016.

This data was pre-parsed with the assistance of `play_neiss`: https://gist.github.com/philshem/3015de98de26df1681635244c95186aa

## Setup 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns
#sns.reset_orig()  # if we don't want to change default matplotlib rc

In [ ]:
import bokeh
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, tools
from bokeh.resources import INLINE

RESOURCES = None
# Use resources=INLINE in bokeh plots to use locally cached Javascript, and render without an internet connection
#RESOURCES = INLINE

In [ ]:
import holoviews as hv
hv.extension('bokeh', 'matplotlib')

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly
init_notebook_mode(connected=True)

In [ ]:
import pandas as pd

## Read data 

In [ ]:
infile = "neiss.csv"

use_fields = "case_id trmt_date age narr sex diag body_part disposition location race_text prod1_text fire_involvement hospital_stratum".split()
categorical_fields = "sex race_text diag body_part disposition location prod1_text fire_involvement hospital_stratum".split()
specified_dtypes = {field:'category' for field in categorical_fields}

data = pd.read_table(infile, usecols=use_fields,
                     parse_dates=['trmt_date'],
                     dtype=specified_dtypes, 
                     index_col=0)

# Add incident_count field to clarify summary statistics
data['incident_count'] = 1

data.shape

In [ ]:
# Create a short product description for each product, similarly for body parts and diagnosis
products = set(data['prod1_text'].unique())
short_products = {s:s.split('(')[0].split(',')[0].strip() for s in products}
data['product'] = data['prod1_text'].apply(lambda p: short_products[p]).astype('category')

body_parts = set(data['body_part'].unique())
short_bodyparts = {s:s.split('(')[0].strip() for s in body_parts}
data['body_part'] = data['body_part'].apply(lambda b: short_bodyparts[b]).astype('category')

# Additionally, collapse all 'Burns' diagnoses
diagnoses = set(data['diag'].unique())
short_diags = {s:s.split('(')[0].strip() for s in diagnoses}
for diag in diagnoses:
    if diag[:5]=='Burns':
        short_diags[diag] = "Burns"
data['diagnosis'] = data['diag'].apply(lambda d: short_diags[d]).astype('category')

In [ ]:
# For this demo, I didn't bother with dates.
# Instead, work with day-from-start, setting the first day to 1
# This is equivalent to day-of-month for our dataset
data['day'] = (data['trmt_date'] - data['trmt_date'].min()).dt.days + 1

In [ ]:
# Restrict our dataset to top few products
NUM_PRODUCTS = 12
top_products = set(data['product'].value_counts()[:NUM_PRODUCTS].index)
data = data[ data['product'].isin(top_products) ]
data['product'].cat.remove_unused_categories(inplace=True)

In [ ]:
pd.set_option('display.max_colwidth', -1)
data.head(3)

## Matplotlib 

In [ ]:
# Default figure size
FIGSIZE = (7,4.5)
plt.rcParams['figure.figsize'] = FIGSIZE

In [ ]:
product_counts = data.groupby('product')['incident_count'].aggregate(sum).sort_values()

#with plt.style.context('classic'):
plt.barh(bottom=list(range(len(product_counts))), 
         width=product_counts,
         tick_label=product_counts.index)


In [ ]:
product_counts = data.groupby('product')['incident_count'].aggregate(sum).sort_values()

with plt.style.context('classic'):
    plt.bar(left=list(range(len(product_counts))), 
             height=product_counts,
             tick_label=product_counts.index)


## Seaborn 

In [ ]:
with sns.axes_style('darkgrid'):
    sns.barplot(data=data, x='product', y='incident_count', estimator=sum)#, hue="sex")

In [ ]:
with sns.axes_style('darkgrid'):
    g = sns.barplot(data=data, x='product', y='incident_count', estimator=sum, hue="sex")
    g.figure.autofmt_xdate()

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
table = data.pivot_table(index='body_part', columns='product', 
                         values='incident_count', aggfunc=sum, fill_value=0)
sns.heatmap(table, cmap='Greens', ax=ax)   # table-10 demonstrates a diverging colour scale - seaborn uses automatically if there are negative numbers

In [ ]:
sns.boxplot(data=data, y='product', x='age')

In [ ]:
# if using kind='scatter', try joint_kws={'alpha': 0.2} to pass an argument down to the 
# matplotlib scatter call (scatter will still not be a great choice)
sns.jointplot(data=data, x='day', y='age', kind='hex')

## Bokeh 

In [ ]:
# A basic scatter plot - will have pan and zoom

subset = data.sample(200)

p = figure(width=400, height=300, x_axis_label="day", y_axis_label="patient age")

p.circle(subset['day'], subset['age'], size=10, alpha=0.7)

output_notebook(resources=RESOURCES)
show(p)

In [ ]:
# Add a legend with click policy 'hide', and a hover tool 

subset = data.sample(200)

# Bokeh probably has a better way to do this than two separate ColumnDataSource objects
source_male = ColumnDataSource(subset[subset['sex']=='Male']["day age narr".split()] )
source_female = ColumnDataSource(subset[subset['sex']=='Female']["day age narr".split()] )

p = figure(width=400, height=300, x_axis_label="day", y_axis_label="patient age")

p.circle(source=source_male, x='day', y='age', color='teal', alpha=0.7, size=10, legend='Male')
p.circle(source=source_female, x='day', y='age', color='darkorange', alpha=0.7, size=10, legend='Female')

p.add_tools(tools.HoverTool(tooltips=[('Narrative', '@narr')]))

p.legend.click_policy='hide'

output_notebook(resources=RESOURCES)
show(p)

In [ ]:
# Two plots with linked axes

subset = data.sample(500)

source1 = ColumnDataSource(subset[subset['product']=='Basketball']["day age narr".split()] )
source2 = ColumnDataSource(subset[subset['product']=='Stairs or steps']["day age narr".split()] )

p1 = figure(width=300, height=300, title='Basketball',
            x_axis_label="day", y_axis_label="patient age")

p1.circle(source=source1, x='day', y='age', size=10, alpha=0.7)

p2 = figure(width=300, height=300, title='Stairs',
            x_axis_label="day", y_axis_label="patient age",
            x_range=p1.x_range, y_range=p1.y_range)

p2.circle(source=source2, x='day', y='age', size=10, alpha=0.7)

l = bokeh.layouts.row(p1,p2)

output_notebook(resources=RESOURCES)
show(l)

In [ ]:
# Two plots, linked sources for selection
# (also linked age axis)

subset = data.sample(500)

source = ColumnDataSource(subset["day age narr".split()] )
source.add(subset['product'].astype('str'), 'product')

product_list = list(subset['product'].unique())

p1 = figure(width=300, height=300,
            tools="box_zoom,box_select,lasso_select,reset",
            x_axis_label="day", y_axis_label="patient age")
p1.circle(source=source, x='day', y='age', 
          size=10, alpha=0.4, 
          selection_color='red', selection_fill_alpha=0.4)

p2 = figure(width=400, height=300, 
            tools="box_zoom,box_select,lasso_select,reset",
            y_range=product_list, x_range = p1.x_range,
            x_axis_label="day")
p2.circle(source=source, x='day', y='product', 
          size=10, alpha=0.4, 
          selection_color='red', selection_fill_alpha=0.4)

l = bokeh.layouts.row(p1,p2)

output_notebook(resources=RESOURCES)
show(l)

## Holoviews

In [ ]:
# If you want to use less points, replace data with e.g. data.sample(2000)

ds = hv.Dataset(data, kdims=["sex","product","body_part"],
                      vdims=["age","day","incident_count","narr"])

In [ ]:
ds

In [ ]:
%%output backend='bokeh' 
%%opts Scatter (size=5 alpha=0.5)

scatter = ds.reindex(kdims=['product'], vdims=['age', 'day']).to(hv.Scatter, "day", "age") 
scatter   #.layout('product')

In [ ]:
%%output backend='bokeh'
%%opts Scatter [tools=['hover']] (size=5 alpha=0.5)

# Scatter plot with hover tool that includes narrative
scatter = ds.to(hv.Scatter, "day", ["age","narr"]).grid('sex')

scatter

In [ ]:
%%output backend='bokeh'
%%opts Bars [xrotation=90 tools=['hover']]
%%opts Scatter [tools=['hover']] (size=5 alpha=0.5)

# the above is ok, but we'd like to know how many incidents are in each body_part for current product - add a bar plot
# retain incident_count as our value dimension, then aggregate it over body_part and product, then specify a bar plot over body_part
bars = ds.reindex(kdims=["product","body_part"],vdims=['incident_count']).aggregate(['body_part', 'product'], function=sum).to(hv.Bars, kdims=['body_part'], vdims=['incident_count'])

scatter = ds.to(hv.Scatter, "day", ["age","narr"]).overlay('sex')

bars + scatter

In [ ]:
%%output backend='bokeh' 
%%opts HeatMap [colorbar=True width=600 tools=['hover'] toolbar='above' xrotation=45] (cmap='Blues')

# A heatmap with a hover tool showing vdim and kdim values
# Our first HeatMap vdim, incident_count, is used to colour the heatmap, but our second, mean age, is available to the tooltip too
aggregated_values = ds.reindex(kdims=["body_part","product"], vdims=["incident_count", "age"]).aggregate(["body_part","product"], function={'incident_count':'sum', 'age':'mean'})
aggregated_values.to(hv.HeatMap, kdims=["body_part","product"], vdims=["incident_count", "age"])

## Plotly 

In [ ]:
subset = data.sample(200)

trace = go.Scatter(x=subset['day'], y=subset['age'], 
                   mode='markers',
                   marker={'size': 10, 'opacity': 0.7})

iplot([trace])

In [ ]:
subset = data.sample(200)

male = subset[ subset['sex']=='Male' ]
female = subset[ subset['sex']=='Female']

trace1 = go.Scatter(x=male['day'], y=male['age'], text=male['narr'],
               mode='markers',
               marker={'size': 10, 'opacity': 0.7},
               name='Male')
trace2 = go.Scatter(x=female['day'], y=female['age'], text=female['narr'],
               mode='markers',
               marker={'size': 10, 'opacity': 0.7},
               name='Female')

layout = go.Layout(xaxis={'title':'Day of month'},
                   yaxis={'title':'Patient age'},
                   hovermode='closest')

fig = go.Figure(data=[trace1, trace2], layout=layout)

iplot(fig)

In [ ]:
trace1 = go.Histogram(x=data[data['sex']=='Male']['age'], 
                      opacity=0.7,
                      name='Male')

trace2 = go.Histogram(x=data[data['sex']=='Female']['age'], 
                      opacity=0.7,
                      name='Female')

layout = go.Layout(barmode='overlay')
fig = go.Figure(data=[trace1, trace2], layout=layout)

iplot(fig)

In [ ]:
trace = go.Box(x = data['diagnosis'], y=data['age'])

layout = {
    'yaxis': {'rangemode': 'tozero',
              'title': 'Patient age'}
}

fig = {'data': [trace], 'layout': layout}

iplot(fig)

In [ ]:
table = data.pivot_table(index='diagnosis', columns='product', 
                         values='incident_count', aggfunc=sum, fill_value=0)

trace = go.Heatmap(z=table.as_matrix(), 
                     x=table.columns, y=table.index,
                     colorscale='Greens',
                     reversescale=True)

iplot([trace])

In [ ]:
subset = data.sample(300)

trace = go.Scatter3d(x=subset['location'], y=subset['product'], z=subset['age'], 
                     mode='markers',
                     marker={'opacity': 0.7},
                     text=subset['narr'])

iplot([trace])